<a href="https://colab.research.google.com/github/ParalelaUCM/biciMAD/blob/master/triciclos2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

!pip install pyspark

In [0]:
import json
from pyspark import SparkContext
sc = SparkContext()

In [0]:
rdd_usages = {}
def mapper_usages(line):
  data = json.loads(line)
  user = data['user_type']
  user_day = data['user_day_code']
  start = data['idunplug_station']
  end = data['idplug_station']
  date = data['unplug_hourTime']['$date'][0:10]
  hora = data['unplug_hourTime']['$date'][11:19]
  time = data['travel_time']
  age = data['ageRange']
  try:
    track = data['track']
  except:
    track = None
  return {"user_type": user,
          "user_day_code": user_day,
          "start": start,
          "end": end,
          "travel_time": time,
          "date": date,
          "hour": hora,
          "age": age,
          "track": track}

In [85]:
directory = '/content'
rdd_usages['2019'] = sc.emptyRDD()
for filename in os.listdir(directory):
    if filename.endswith(".json"):
      #Nos quedamos con la fecha del dataset en formato YYYYMM
      name = filename.split("_")[0]
      rdd_usages[name] = sc.textFile(os.path.join(directory, filename)).map(mapper_usages)
      rdd_usages['2019'] = rdd_usages['2019'].union(rdd_usages[name])
      #DEBUG starts
      print(name)
      #DEBUG ends
    else:
        continue

201901


In [0]:
rddprima = rdd_usages['2019'].map(lambda x: ((x['user_day_code'],x['user_type']),[x['start'],x['end']]))\
                              .groupByKey().map(lambda x: (x[0],list(x[1])))

In [0]:
def adic(ldl):
  d = {}
  for elem in ldl:
    key,value = str(elem[0]),str(elem[1])
    if key in d:
      d[key].append(value)
    else:
      d[key]=[value]
  return d

In [0]:
separador = "_"
def unomas(dic):
  dk = list(dic.keys())
  for key in dk:
    if not isinstance(key, tuple): #ya es un ciclo
      for elem in dic[key]:      
        if key==elem:
          dic[tuple(set([key]))]=1 #[integer] indica ciclo longitud 1
        else:
          spl = key.split(separador)
          if elem in spl:
            pos = spl.index(elem)
            lis = spl[pos:]
            dic[tuple(set(lis))] = len(spl)-pos #[integer] indica ciclo
          else:
            try:
              dic[key+separador+elem]=dic[elem]
            except KeyError:
              pass
  return dic

In [0]:
def sigue(dic):
  if len(dic) == len(unomas(dic)):
    return dic
  else:
    return (sigue(unomas(dic)))

In [0]:
def tieneciclos(dic):
  resultado = False
  for key,value in dic.items():
    if isinstance(key,tuple):
      resultado = True
  return resultado


In [0]:
def dameciclos(dic):
  dic2 = {}
  for key,value in dic.items():
    if isinstance(key,tuple):
      try:
        dic2[value].append(key)
      except:
        dic2[value]=[key]
  return dic2

In [0]:
rdd_ciclos = rddprima.mapValues(lambda x: adic(x)).mapValues(lambda x: sigue(x))\
          .filter(lambda x: len(x[1])>0).filter(lambda x: tieneciclos(x[1]))\
          .mapValues(lambda x:dameciclos(x))

In [104]:
for m in range(13): # para ver ciclos de qué longitudes hemos encontrado (hasta 10)
  print(m,not(rdd_ciclos.filter(lambda x: m in x[1].keys()).mapValues(lambda x: x[m]).isEmpty()))

0 False
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 False
9 False
10 True
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
21 False
22 False
23 False
24 False


In [0]:
rdd_triciclos = rdd_ciclos.filter(lambda x: 3 in x[1].keys()).mapValues(lambda x: x[3])
rdd_biciclos = rdd_ciclos.filter(lambda x: 2 in x[1].keys()).mapValues(lambda x: x[2])

In [106]:
print("GENERAL")
for i in range(13):
  aux = rdd_ciclos.filter(lambda x: i in x[1].keys()).mapValues(lambda x: x[i])
  print("ciclos de longitud",i,":",aux.count())

GENERAL
ciclos de longitud 0 : 0
ciclos de longitud 1 : 6100
ciclos de longitud 2 : 24069
ciclos de longitud 3 : 1552
ciclos de longitud 4 : 174
ciclos de longitud 5 : 33
ciclos de longitud 6 : 5
ciclos de longitud 7 : 2
ciclos de longitud 8 : 0
ciclos de longitud 9 : 0
ciclos de longitud 10 : 1
ciclos de longitud 11 : 0
ciclos de longitud 12 : 0


In [107]:
print("TRABAJADORES")
for i in range(13):
  aux = rdd_ciclos.filter(lambda x: x[0][1]==3).filter(lambda x: i in x[1].keys()).mapValues(lambda x: x[i])
  print("ciclos de longitud",i,":",aux.count())

TRABAJADORES
ciclos de longitud 0 : 0
ciclos de longitud 1 : 587
ciclos de longitud 2 : 7
ciclos de longitud 3 : 0
ciclos de longitud 4 : 2
ciclos de longitud 5 : 0
ciclos de longitud 6 : 0
ciclos de longitud 7 : 0
ciclos de longitud 8 : 0
ciclos de longitud 9 : 0
ciclos de longitud 10 : 0
ciclos de longitud 11 : 0
ciclos de longitud 12 : 0
